# Procesamiento del lenguaje natural. NLP 🙃

En este jupyter notebook se encuentra todo el proceso de sentimental analysis de los datos disponibles en la API y almacenados en el dataframe de UN General Debates ya tratado y procesado. (Véase 1.Database.ipynb)

## Índice 📎
- Importación de librerias y funciones
- Importación del dataset
- NLP
>- Tokenizar y Stop Words
>- Sentiment analysis
- Conclusiones
>- Índice global de la felicidad

## 1. Importación de librerias y funciones 📚

In [1]:
import pandas as pd
import string
import spacy
import nltk
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import os

## 2. Importación del dataset 📖

In [2]:
data = pd.read_csv("Data/un_to_csv")
#data.head()

In [3]:
data2 = data.drop("Unnamed: 0", axis=1)
data2.head()

,Year,Country,Speech
0,1989,MDV,it is indeed a pleasure for me and the members...
1,1989,FIN,"may i begin by congratulating you sir, on you..."
2,1989,NER,"mr president, it is a particular pleasure for..."
3,1989,URY,during the debate at the fortieth session of t...
4,1989,ZWE,i should like at the outset to express my dele...


## 3. NLP 🫀

###  Tokenizar y Stop Words 🚦

Las stop words, se definen como las palabras vacías, es decir, sin significado como pronombres, artículos o preposiciones; las cuales vamos a filtrar antes de procesar los datos.

Para normalizar nuestros tokens emplearemos el stremming que consiste en quitar y reemplazar los sufijos de la raíz de la palabra, para ello escribiremos una función que tokenizará los speechs uniendolo a la "eliminación" de las stops words:

***Fuentes empleadas:***
- [How to tokenize a string sentence in NLTK](https://www.kite.com/python/answers/how-to-tokenize-a-string-sentence-in-nltk)
- [How do I tokenize a string sentence in NLTK?](https://stackoverflow.com/questions/15057945/how-do-i-tokenize-a-string-sentence-in-nltk)

In [4]:
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/olayamarquesgomez/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
def tokenize(x):
    tok = RegexpTokenizer(r'\w+')
    tokens = tok.tokenize(x)
    return tokens

In [6]:
data2["Speech"] = data2.Speech.apply(tokenize)
data2.head()

,Year,Country,Speech
0,1989,MDV,"[it, is, indeed, a, pleasure, for, me, and, th..."
1,1989,FIN,"[may, i, begin, by, congratulating, you, sir, ..."
2,1989,NER,"[mr, president, it, is, a, particular, pleasur..."
3,1989,URY,"[during, the, debate, at, the, fortieth, sessi..."
4,1989,ZWE,"[i, should, like, at, the, outset, to, express..."


A continuación procedemos a eliminar las stop words, para la cual también crearemos una función:

***Fuentes empleadas:***
- [Removing stop words in NLTK in Python](https://www.geeksforgeeks.org/removing-stop-words-nltk-python/)

In [7]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/olayamarquesgomez/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
def stop_words(x):
    stop_words = set(stopwords.words("english"))
    filtrada = []
    for string in x:
        if string not in stop_words:
            filtrada.append(string)
    return " ".join(filtrada)

In [9]:
data2["Speech"] = data2.Speech.apply(stop_words)
data2.head()

,Year,Country,Speech
0,1989,MDV,indeed pleasure members delegation extend amba...
1,1989,FIN,may begin congratulating sir election presiden...
2,1989,NER,mr president particular pleasure behalf delega...
3,1989,URY,debate fortieth session general assembly four ...
4,1989,ZWE,like outset express delegation satisfaction pl...


### Sentiment analysis 💞

Tras haber preparado nuestros datos procedemos a realizar el análisis de sentimientos, de nuevo emplearemos una función:

In [10]:
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/olayamarquesgomez/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [11]:
def sentimentAnalysis(x):
    sia = SentimentIntensityAnalyzer()
    polaridad = sia.polarity_scores(x)
    pol = polaridad["compound"]
    return pol

In [12]:
#creamos una nueva columna que imprima la polaridad de todos los speechs
data2["Polarity"] = data2.Speech.apply(sentimentAnalysis)
data2.head()

,Year,Country,Speech,Polarity
0,1989,MDV,indeed pleasure members delegation extend amba...,0.9998
1,1989,FIN,may begin congratulating sir election presiden...,0.9998
2,1989,NER,mr president particular pleasure behalf delega...,0.9999
3,1989,URY,debate fortieth session general assembly four ...,0.9996
4,1989,ZWE,like outset express delegation satisfaction pl...,0.9999


Una vez analizada la polaridad vamos a agrupar los países para comprobar quién es más positivo y negativo en sus speechs:

In [13]:
data2.Country.value_counts()

SDN    46
IRQ    46
TUN    46
NZL    46
NLD    46
       ..
VAT    13
TLS    13
MNE    10
SSD     5
EU      5
Name: Country, Length: 199, dtype: int64

In [14]:
#país
analisis = data2.groupby(["Country"])["Polarity"].mean().sort_values().to_frame().reset_index()
analisis

,Country,Polarity
0,ISR,0.525035
1,SYR,0.610915
2,CUB,0.614974
3,IRN,0.669939
4,SGP,0.688682
...,...,...
194,FJI,0.999760
195,KOR,0.999776
196,THA,0.999784
197,CHN,0.999791


In [15]:
analisis.max()

Country          ZWE
Polarity    0.999836
dtype: object

In [16]:
analisis.min()

Country          AFG
Polarity    0.525035
dtype: object

In [17]:
#año
analisis2 = data2.groupby(["Year"])["Polarity"].mean().sort_values().to_frame().reset_index()
#analisis2

In [18]:
analisis2.max()

Year        2015.000000
Polarity       0.999825
dtype: float64

In [19]:
analisis2.min()

Year        1970.000000
Polarity       0.720027
dtype: float64

## Conclusiones 🥸

En relación con los "speechs" pronunciados durante los debates de la UN se concluye que:
    
- El país más positivo es Zimbawe, mientras que el más negativo es Afganistán
- El año 1970 fue el más negativo, siendo 2015 el año medido más positivo

### Índice global de felicidad 😁

El índice global de felicidad es una publicación anual de las Naciones Unidas que mide la felicidad en 157 países, basándose en diversos factores, como el PIB per cápita. 

***¿Coincidirá con las conclusiones analizadas de los speechs de los debates?***

Para comprobarlo vamos a emplear la herramienta de webscrapping y extraer la tabla con el índice global de felicidad descargado de la [wikipedia](https://en.wikipedia.org/wiki/World_Happiness_Report).

#### Toma de datos 🎣

In [20]:
url = "https://en.wikipedia.org/wiki/World_Happiness_Report"

In [21]:
html = requests.get(url)

In [22]:
soup = BeautifulSoup(html.content, "html.parser")

In [23]:
#document.querySelectorAll("Table").forEach(elm => elm.style.background = 'red')
table = soup.find_all('table',{'class':'wikitable sortable'})[0]

In [24]:
#reorganización de la tabla
rows = table.find_all("tr")
rows = [row.text.replace("\n\n", ",").replace("\n", ",").replace("\xa0", "").strip(",").split(",") for row in rows]

In [25]:
#creación del dataframe
nombre_columnas = rows[0]
data = rows[1:]
felicidad = pd.DataFrame(data, columns=nombre_columnas)
felicidad.head(10)

,Overall rank,Country or region,Score,GDP per capita,Social support,Healthy life expectancy,Freedom to make life choices,Generosity,Perceptions of corruption
0,1,Finland,7.809,1.285,1.500,0.961,0.662,0.160,0.478
1,2,Denmark,7.646,1.327,1.503,0.979,0.665,0.243,0.495
2,3,Switzerland,7.560,1.391,1.472,1.041,0.629,0.269,0.408
3,4,Iceland,7.504,1.327,1.548,1.001,0.662,0.362,0.145
4,5,Norway,7.488,1.424,1.495,1.008,0.670,0.288,0.434
5,6,Netherlands,7.449,1.339,1.464,0.976,0.614,0.336,0.369
6,7,Sweden,7.353,1.322,1.433,0.986,0.650,0.273,0.442
7,8,New Zealand,7.300,1.242,1.487,1.008,0.647,0.326,0.461
8,9,Austria,7.294,1.317,1.437,1.001,0.603,0.256,0.281
9,10,Luxembourg,7.238,1.537,1.388,0.986,0.610,0.196,0.367


In [26]:
#reenombramos las columnas
felicidad.columns = ["Ranking", "País", "Puntuación", "PIB", "Apoyo social", "Esperanza de vida", "Libertad", "Generosidad", "Corrupción"]

In [32]:
#eliminamos las columnas cuya información no es relevante
felicidad2 = felicidad.drop(["PIB", "Apoyo social", "Esperanza de vida", "Libertad", "Generosidad", "Corrupción"], axis=1)
felicidad2.head()

,Ranking,País,Puntuación
0,1,Finland,7.809
1,2,Denmark,7.646
2,3,Switzerland,7.560
3,4,Iceland,7.504
4,5,Norway,7.488


#### Comparación 👩🏼‍💻

In [30]:
zimbabue = felicidad2[felicidad2["País"] == "Zimbabwe"]
zimbabue

,Ranking,País,Puntuación
150,151,Zimbabwe,3.299


In [31]:
afganistan = felicidad2[felicidad2["País"] == "Afghanistan"]
afganistan

,Ranking,País,Puntuación
152,153,Afghanistan,2.567


#### Resultados 💬

Resulta curioso en el caso de Zimbawe que siendo el país cuyos speechs son más positivos se encuentre de los últimos países en el ranking de felicidad. Ello también puede darse debido a lo medido valorizado como aquellos índices que proporcionan felicidad (pib, corrupción, esperanza de vida...etc).

En el caso de Afganistán se encuentra el último en la lista ("país más infeliz"), lo cual coincide con la negatividad de sus speechs. 

Desde el punto de vista de Finlandia considerado el país más feliz del mundo... Su polaridad es de:

In [33]:
finlandia = analisis[analisis["Country"] == "FIN"]
finlandia

,Country,Polarity
142,FIN,0.999367


Contrastando que sus speechs también son muy positivos.